In [1]:
! pip install mlxtend

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [3]:
df_train=pd.read_csv("house_price_train.csv")
df_train.drop("Id",axis=1,inplace=True) ## reduntatnt
output_col=df_train['SalePrice']
df_train=df_train.drop('SalePrice',axis=1)

In [4]:
df_train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [7]:
df_train.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 80, dtype: object

In [11]:
# Deleting columns which have mnore than 50% NULL values
# null_cols=[]
# for col in df_train.columns:
#     if df_train[col].isnull().sum()>=(0.5*len(df_train)):
#         null_cols.append([col,df_train[col].isnull().sum()])
# #         print(col,df[col].isnull().sum())

In [12]:
# null_cols=[val[0] for val in null_cols]

In [13]:
# df_train.drop(null_cols,axis=1,inplace=True)

In [ ]:

def column_with_most_reduntant_null_values(threshold,df_train:pd.DataFrame):
    null_cols=[]
    for col in df_train.columns:
        if df_train[col].isnull().sum()>=(threshold*len(df_train)):
            null_cols.append(col)
    df_train.drop(null_cols,axis=1,inplace=True)

In [ ]:
column_with_most_reduntant_null_values(threshold=0.5,df_train=df_train)

In [14]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,112,0,0,0,0,4,2010,WD,Normal,142125


In [15]:
df_train.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF'

In [16]:
i=0
for cols in df_train.columns:
    if df_train[cols].dtype=="int64":
        print(cols,i)
#         i+=1
    elif df_train[cols].dtype=="object":
#         print(cols,i)
        i+=1
    elif df_train[cols].dtype=="float64":
        print(cols,i)
#         i+=1
print(i)

MSSubClass 0
LotFrontage 1
LotArea 1
OverallQual 12
OverallCond 12
YearBuilt 12
YearRemodAdd 12
MasVnrArea 17
BsmtFinSF1 24
BsmtFinSF2 25
BsmtUnfSF 25
TotalBsmtSF 25
1stFlrSF 29
2ndFlrSF 29
LowQualFinSF 29
GrLivArea 29
BsmtFullBath 29
BsmtHalfBath 29
FullBath 29
HalfBath 29
BedroomAbvGr 29
KitchenAbvGr 29
TotRmsAbvGrd 30
Fireplaces 31
GarageYrBlt 33
GarageCars 34
GarageArea 34
WoodDeckSF 37
OpenPorchSF 37
EnclosedPorch 37
3SsnPorch 37
ScreenPorch 37
PoolArea 37
MiscVal 37
MoSold 37
YrSold 37
SalePrice 39
39


In [17]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,112,0,0,0,0,4,2010,WD,Normal,142125


In [ ]:
def fill_null_values(for_numerical_cols:str,for_categorical_cols:str,df_train):
    for cols in df_train.columns:
        if df_train[cols].dtype=="int64" or df_train[cols].dtype=="float64":
            if for_numerical_cols=="median":
                median_=df_train[cols].median()
                df_train[cols].fillna(median_,inplace=True)
        elif df_train[cols].dtype=="object":
            if for_categorical_cols=="most_frequent":
                d=list(df_train[cols].value_counts().index) # most frequent
                df_train[cols].fillna(d[0],inplace=True)

In [ ]:
fill_null_values(for_numerical_cols="median",for_categorical_cols="most_frequent",df_train=df_train)

In [19]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>(0):

        print(col,df[col].isnull().sum(),df_train[col].dtypes)


In [20]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [ ]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [21]:
len(continuous_cols)

37

In [22]:
continuous_cols

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SalePrice']

In [24]:
categorical_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [25]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(20, 20)

# for ax, col in zip(axes, continuous_cols):
#     print()
#     sns.boxplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [26]:
df_train.loc[:,continuous_cols]

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,61,0,0,0,0,0,2,2008,208500
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,298,0,0,0,0,0,0,5,2007,181500
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,42,0,0,0,0,0,9,2008,223500
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,35,272,0,0,0,0,2,2006,140000
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,40,0,0,0,0,0,8,2007,175000
1456,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,349,0,0,0,0,0,0,2,2010,210000
1457,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,0,60,0,0,0,0,2500,5,2010,266500
1458,20,68.0,9717,5,6,1950,1996,0.0,49,1029,...,366,0,112,0,0,0,0,4,2010,142125


In [27]:
# plt.figure(figsize=(20,15))
# sns.heatmap(df_train.loc[:,continuous_cols].corr(method='spearman'),annot=True,cmap="RdYlGn")
# plt.show()

Deleted ouliers using IQR

In [ ]:
df_train=pd.concat([output_col,df_train],axis=1) # since rows will be deleted here


In [29]:
# Outlier delete

for col in continuous_cols:
    q1=df_train[col].quantile(0.25)
    q3=df_train[col].quantile(0.75)
    iqr=q3-q1
    l=q1-1.5*iqr
    h=q3+1.5*iqr
    print("[",l,",",h,"]")
    df_train = df_train[(df_train[col] <= h)] 
    df_train = df_train[(df_train[col] >=l)] 



[ -55.0 , 145.0 ]
[ 30.0 , 110.0 ]
[ 2440.0 , 16936.0 ]
[ 2.0 , 10.0 ]
[ 3.5 , 7.5 ]
[ 1887.0 , 2071.0 ]
[ 1909.0 , 2061.0 ]
[ -256.5 , 427.5 ]
[ -1066.5 , 1777.5 ]
[ 0.0 , 0.0 ]
[ -587.5 , 1728.5 ]
[ 132.0 , 1948.0 ]
[ 201.5 , 2021.5 ]
[ -1111.5 , 1852.5 ]
[ 0.0 , 0.0 ]
[ 253.0 , 2597.0 ]
[ -1.5 , 2.5 ]
[ 0.0 , 0.0 ]
[ -0.5 , 3.5 ]
[ -1.5 , 2.5 ]
[ 0.5 , 4.5 ]
[ 1.0 , 1.0 ]
[ 2.0 , 10.0 ]
[ -1.5 , 2.5 ]
[ 1904.875 , 2061.875 ]
[ -0.5 , 3.5 ]
[ -13.25 , 928.75 ]
[ -237.0 , 395.0 ]
[ -99.0 , 165.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.5 , 12.5 ]
[ 2004.0 , 2012.0 ]
[ 21950.0 , 330750.0 ]


In [ ]:
output_col=df_train['SalePrice']
df_train.drop("SalePrice",axis=1,inplace=True)

In [ ]:
def encoding_categorical_cols(df_train:pd.DataFrame,categorical_columns:list,strategy:str):
    df_categorical=pd.DataFrame(df_train.loc[:,categorical_columns])
    if strategy=="labelencoder":
        for col in list(df_categorical.columns):
            cat=[]
            d={}

            cat.append([df_categorical[col].value_counts().index,len(df_categorical[col].value_counts().index)])

            for j in range(int(cat[0][1])):
                d[str(cat[0][0][j])]=j
            df_categorical[col]=df_categorical[col].map(d)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        
        df_train=pd.concat([df_train,df_categorical],axis=1)
        
    if strategy=="onehotencoder":
        one_hot_encoded=pd.get_dummies(df_categorical)
        # print(one_hot_encoded)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    return df_train

In [ ]:
df_train=encoding_categorical_cols(df_train=df_train,categorical_columns=categorical_cols,strategy="labelencoder")

In [36]:
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,0,0,0,0,0,0,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,1,0,0,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,1,0,0,0,0,0,0,0
6,20,75.0,10084,8,5,2004,2005,186.0,1369,0,...,0,0,0,0,0,0,0,0,0,0
10,20,70.0,11200,5,5,1965,1965,0.0,906,0,...,1,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,20,63.0,8500,7,5,2004,2004,106.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1448,50,70.0,11767,4,7,1910,2000,0.0,0,0,...,1,1,0,1,1,1,0,0,0,0
1451,20,78.0,9262,8,5,2008,2009,194.0,0,0,...,2,0,0,0,2,0,0,0,1,1
1454,20,62.0,7500,7,5,2004,2005,0.0,410,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
df_train.loc[:,continuous_cols].describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.000000,548.0,...,548.000000,548.000000,548.0,548.0,548.0,548.0,548.0,548.000000,548.000000,548.000000
mean,49.105839,68.379562,9100.343066,6.231752,5.368613,1983.169708,1989.204380,74.423358,429.483577,0.0,...,90.144161,40.246350,0.0,0.0,0.0,0.0,0.0,6.324818,2007.784672,177890.208029
std,31.660674,13.376085,2593.927851,1.235939,0.707159,25.252264,19.607907,106.259227,401.742272,0.0,...,97.322300,43.943285,0.0,0.0,0.0,0.0,0.0,2.658566,1.322420,54472.843763
min,20.000000,30.000000,2887.000000,2.000000,4.000000,1910.000000,1950.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,2006.000000,37900.000000
25%,20.000000,60.750000,7683.000000,5.000000,5.000000,1965.000000,1972.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,5.000000,2007.000000,136975.000000
50%,50.000000,69.000000,9000.000000,6.000000,5.000000,1995.500000,1999.000000,0.000000,422.500000,0.0,...,99.000000,33.000000,0.0,0.0,0.0,0.0,0.0,6.000000,2008.000000,175700.000000
75%,60.000000,75.000000,10676.750000,7.000000,6.000000,2004.000000,2005.000000,143.000000,725.000000,0.0,...,165.000000,63.000000,0.0,0.0,0.0,0.0,0.0,8.000000,2009.000000,212225.000000
max,120.000000,109.000000,16770.000000,10.000000,7.000000,2009.000000,2010.000000,420.000000,1619.000000,0.0,...,379.000000,162.000000,0.0,0.0,0.0,0.0,0.0,12.000000,2010.000000,328900.000000


In [38]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>0:
        print(col)

In [39]:
### SCALING

In [40]:
scaler=scaler=MinMaxScaler()
df_train=pd.DataFrame(scaler.fit_transform(df_train),columns=df_train.columns,)
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,0.4,0.443038,0.400706,0.625,0.333333,0.939394,0.883333,0.466667,0.436072,0.0,...,0.000000,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.000,0.00
1,0.4,0.481013,0.602391,0.625,0.333333,0.919192,0.866667,0.385714,0.300185,0.0,...,0.000000,0.00,0.25,0.000000,0.0,0.0,0.0,0.0,0.000,0.00
2,0.4,0.683544,0.819203,0.750,0.333333,0.909091,0.833333,0.833333,0.404571,0.0,...,0.000000,0.00,0.25,0.000000,0.0,0.0,0.0,0.0,0.000,0.00
3,0.0,0.569620,0.518404,0.750,0.333333,0.949495,0.916667,0.442857,0.845584,0.0,...,0.000000,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.000,0.00
4,0.0,0.506329,0.598790,0.375,0.333333,0.555556,0.250000,0.000000,0.559605,0.0,...,0.333333,0.00,0.00,0.333333,0.5,0.0,0.0,0.0,0.000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,0.0,0.417722,0.404307,0.625,0.333333,0.949495,0.900000,0.252381,0.000000,0.0,...,0.000000,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.000,0.00
544,0.3,0.506329,0.639631,0.250,1.000000,0.000000,0.833333,0.000000,0.000000,0.0,...,0.333333,0.25,0.00,0.333333,0.5,0.5,0.0,0.0,0.000,0.00
545,0.0,0.607595,0.459195,0.750,0.333333,0.989899,0.983333,0.461905,0.000000,0.0,...,0.666667,0.00,0.00,0.000000,1.0,0.0,0.0,0.0,0.125,0.25
546,0.0,0.405063,0.332277,0.625,0.333333,0.949495,0.916667,0.000000,0.253243,0.0,...,0.000000,0.00,0.00,0.000000,0.0,0.0,0.0,0.0,0.000,0.00


In [41]:
from sklearn.metrics import r2_score

In [42]:
X=df_train
Y=output_col

In [43]:
accuracies={}

In [44]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=None,shuffle=True)


In [ ]:
def sf_selector(algo:str,forward:bool,floating:bool,scoring:int,X_train,Y_train,X_test,Y_test,min_featues:int,cv:int,verbose:int):
    model=eval(algo)
    sfs=SFS(model, 
           k_features=(min_featues,len(X_train.columns)), 
           forward=forward, 
           floating=floating, 
           verbose=verbose,
           scoring=scoring,
           cv=cv,
           n_jobs=-1)
    sfs=sfs.fit(X_train, Y_train)
    print("Selected Features",sfs.k_feature_names_)
    model_=eval(algo)
    model_.fit(X_train.loc[:,list(sfs.k_feature_names_)],Y_train)
    score=model_.score(X_test.loc[:,list(sfs.k_feature_names_)],Y_test)
    
    print(f"Prediction using selected features using {model_} is :",score)
    accuracies[model]=score

In [ ]:
sf_selector(algo="LinearRegression()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="RandomForestRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="SVR()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="KNeighborsRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="DecisionTreeRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="SGDRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="ElasticNet()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="Lasso()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="Ridge()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [3]:

# sfs1 = SFS(LinearRegression(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [46]:
# model_=LinearRegression()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["LinearRegression"]=score

Prediction using selected features is : 0.9108221509034548


In [4]:

# sfs1 = SFS(RandomForestRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [5]:
# model_=RandomForestRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["RandomForestRegressor"]=score

In [6]:
# sfs1 = SFS(SVR(), 
#            k_features=(3,len(X_train.columns)), 
#            forward=True, 
#            floating=False, 
#            verbose=1,
#            scoring='r2',
#            cv=5,
#            n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [7]:
# model_=SVR()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["SVR"]=score

In [8]:
# sfs1 = SFS(KNeighborsRegressor(), 
#            k_features=(3,len(X_train.columns)), 
#            forward=True, 
#            floating=False, 
#            verbose=1,
#            scoring='r2',
#            cv=5,
#            n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [9]:
# model_=KNeighborsRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["KNeighborsRegressor"]=score

In [10]:
# sfs1 = SFS(DecisionTreeRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [11]:
# model_=DecisionTreeRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["DecisionTreeRegressor"]=score

In [12]:
# sfs1 = SFS(SGDRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [13]:
# model_=SGDRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["SGDRegressor"]=score

In [14]:
# sfs1 = SFS(ElasticNet(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [15]:
# model_=ElasticNet()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["ElasticNet"]=score

In [16]:
# sfs1 = SFS(Lasso(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [17]:
# model_=Lasso()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["Lasso"]=score

In [ ]:
# sfs1 = SFS(Ridge(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [62]:
# model_=Ridge()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["Ridge"]=score 

Prediction using selected features is : 0.9112245410876724


In [63]:
accuracies

{'LinearRegression': 0.9108221509034548,
 'RandomForestRegressor': 0.9172228287742473,
 'SVR': 0.8967283895574106,
 'KNeighborsRegressor': 0.8732109314942657,
 'DecisionTreeRegressor': 0.7861653206656223,
 'SGDRegressor': 0.8495881177037554,
 'ElasticNet': -0.001730401062585596,
 'Lasso': -0.001730401062585596,
 'Ridge': 0.9112245410876724}